In [2]:
from graph_of_thoughts.prompter import LanguageModelPrompter
from graph_of_thoughts.language_model import ChatGPT
lm = ChatGPT()
prompter = LanguageModelPrompter(lm, "Main Task: Sort this list using merge sort algorithm")
list = "[9, 6, 7, 7, 2, 0, 2, 2, 3, 5, 0, 9, 2, 2, 4, 4, 5, 2, 5, 1, 2, 8, 3, 8, 3, 9, 6, 0, 4, 2, 2, 3]"
num_split = 4
print(f"Sort this list using merge sort algorithm {list}")




Sort this list using merge sort algorithm [9, 6, 7, 7, 2, 0, 2, 2, 3, 5, 0, 9, 2, 2, 4, 4, 5, 2, 5, 1, 2, 8, 3, 8, 3, 9, 6, 0, 4, 2, 2, 3]


In [3]:
split_prompt = prompter.split_prompt({"state": f"Sort this list using merge sort algorithm\nInput", "num_split": num_split, "current": list})
print(split_prompt)

<Instruction>To divide the current task into exactly 4 subtasks, you can start by splitting the list into two equal parts. Then, split each of these two parts into two more equal parts. This will create 4 subtasks, each containing a portion of the original list. 
Only output the 4 results in format json {"0":..., "1": ...} without any addtional text or thoughts.</Instruction>
Input: [9, 6, 7, 7, 2, 0, 2, 2, 3, 5, 0, 9, 2, 2, 4, 4, 5, 2, 5, 1, 2, 8, 3, 8, 3, 9, 6, 0, 4, 2, 2, 3]
Output:


In [9]:
split_result = lm.get_response_texts(lm.query(split_prompt))[0]
print(split_result)


{
  "0": [9, 6, 7, 7, 2, 0, 2, 2, 3, 5],
  "1": [0, 9, 2, 2, 4, 4, 5, 2, 5, 1],
  "2": [2, 8, 3, 8, 3, 9, 6, 0, 4, 2],
  "3": [2, 3]
}


In [10]:
import json
split_result_dict = json.loads(split_result)

In [11]:
generate_result_dict = {}
for k, v in split_result_dict.items():
    generate_prompt = prompter.generate_prompt({"state": "To divide the current task into exactly 4 subtasks, you can start by splitting the list into two equal parts. Then, split each of these two parts into two more equal parts. This will create 4 subtasks, each containing a portion of the original list.", "current":v})
    print(generate_prompt)
    generate_result_dict[k] = json.loads(lm.get_response_texts(lm.query(generate_prompt))[0])["0"]

    print(generate_result_dict[k])

<Instruction>After dividing the list into exactly 4 subtasks, the next step is to recursively sort each of the subtasks using the merge sort algorithm. This involves dividing each subtask further if it contains more than one element, and then merging the sorted sublists back together in the correct order. 
Only output the result in format json {"0":} without any addtional text or thoughts.</Instruction>
Input: [9, 6, 7, 7, 2, 0, 2, 2, 3, 5]
Output:
[0, 2, 2, 2, 3, 5, 6, 7, 7, 9]
<Instruction>After dividing the list into exactly 4 subtasks, the next step is to recursively sort each of these subtasks using the merge sort algorithm. This involves dividing each subtask further if it contains more than one element, and then merging the sorted subtasks back together in the correct order. 
Only output the result in format json {"0":} without any addtional text or thoughts.</Instruction>
Input: [0, 9, 2, 2, 4, 4, 5, 2, 5, 1]
Output:
[0, 1, 2, 2, 2, 4, 4, 5, 5, 9]
<Instruction>After splitting t

In [12]:
print(generate_result_dict)

{'0': [0, 2, 2, 2, 3, 5, 6, 7, 7, 9], '1': [0, 1, 2, 2, 2, 4, 4, 5, 5, 9], '2': [2], '3': [2, 3]}


In [14]:
aggregate_prompt = prompter.aggregate_prompt({"state": "After dividing the list into exactly 4 subtasks, the next step is to recursively sort each of the subtasks using the merge sort algorithm. This involves dividing each subtask further if it contains more than one element, and then merging the sorted sublists back together in the correct order.", "current":generate_result_dict})


KeyError: 0

<Instruction>Sort the given list using the merge sort algorithm. 
Only output the result in format json {"0":} without any addtional text or thoughts.</Instruction>
Input: {'0': [0, 2, 2, 2, 6, 7, 7, 9], '1': [0, 2, 2, 3, 4, 4, 5, 9], '2': [1, 2, 2, 3, 5, 5, 8, 8], '3': [0, 2, 2, 3, 3, 4, 6, 9]}
Output:


In [ ]:
lm.get_response_texts(lm.query(aggregate_prompt))[0]

'{"0": [0, 0, 1, 2, 2, 2, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9]}'